In [14]:
# The basics
import numpy as np
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, LSTM

# utils
import os
import subprocess

In [2]:
parent_relative_path = ".."

In [3]:
colab = 'google.colab' in str(get_ipython())

In [5]:
if colab:
    print("Colab babe")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

    os.system('cp "/content/gdrive/My Drive/Colab Notebooks/.kaggle/kaggle.json" /root/.kaggle/kaggle.json')
    subprocess.run(["git", "clone", "https://github.com/codefupanda/customer_interaction_summary.git"])
    os.system('cd customer_interaction_summary && make requirements && make data > logs.logs')
    parent_relative_path = "./customer_interaction_summary"

In [6]:
isear = pd.read_csv(parent_relative_path + '/data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [7]:
number_of_classes = len(isear.EMOT.unique())

In [8]:
maxlen = 1000
max_words = 10000

In [9]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(isear['SIT'])
sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9063 unique tokens.


In [11]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [12]:
x_train, x_test, y_train, y_test = train_test_split(data, isear['EMOT'])

In [15]:
SimpleRNN?

## Model creation time

In [25]:
model = Sequential()
model.add(Embedding(max_words, output_dim=50, input_length=maxlen))
model.add(SimpleRNN(50, return_sequences=True))
model.add(Flatten())
model.add(Dense(number_of_classes + 1,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1000, 50)          500000    
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 1000, 50)          5050      
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 400008    
Total params: 905,058
Trainable params: 905,058
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(x_train, to_categorical(y_train),
          epochs=5,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Train on 5749 samples, validate on 1917 samples
Epoch 1/5
5749/5749 [==============================] - 46s 8ms/sample - loss: 0.0601 - accuracy: 0.9849 - val_loss: 3.0139 - val_accuracy: 0.3219
Epoch 2/5
5749/5749 [==============================] - 44s 8ms/sample - loss: 0.0524 - accuracy: 0.9861 - val_loss: 3.2419 - val_accuracy: 0.2926
Epoch 3/5
5749/5749 [==============================] - 45s 8ms/sample - loss: 0.0863 - accuracy: 0.9769 - val_loss: 3.0203 - val_accuracy: 0.3208
Epoch 4/5
5749/5749 [==============================] - 46s 8ms/sample - loss: 0.0821 - accuracy: 0.9793 - val_loss: 3.3015 - val_accuracy: 0.3229
Epoch 5/5
5749/5749 [==============================] - 46s 8ms/sample - loss: 0.0468 - accuracy: 0.9871 - val_loss: 3.4531 - val_accuracy: 0.3312


In [14]:
y_pred = model.predict_classes(x_test)
y_pred

array([4, 4, 4, ..., 4, 4, 4])

In [15]:
confusion_matrix(y_test, y_pred)

array([[  0,   0,   0, 267,   0,   0,   0],
       [  0,   0,   0, 275,   0,   0,   0],
       [  0,   0,   0, 265,   0,   0,   0],
       [  0,   0,   0, 280,   0,   0,   0],
       [  0,   0,   0, 298,   0,   0,   0],
       [  0,   0,   0, 269,   0,   0,   0],
       [  0,   0,   0, 263,   0,   0,   0]])